In [1]:
# main.py
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from operator import itemgetter

load_dotenv()

# Inicializar LLM de alta velocidade
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

# Inicializar o Retriever para buscar informações no Pinecone
index_name = "assistente-viagem"
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever()

/Users/iagocampista/anaconda3/envs/travelAgent/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [2]:
# Prompt Template para Info Local
local_info_prompt = PromptTemplate(
    template="""Você é um assistente de viagens muito prestativo.
    Use o CONTEXTO fornecido para responder à PERGUNTA do usuário.
    Se a informação não estiver no contexto, diga que você não sabe. Não invente.

    CONTEXTO: {context}
    PERGUNTA: {question}
    RESPOSTA:""",
    input_variables=["context", "question"],
)

# Criar a Cadeia (Chain)
local_info_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | local_info_prompt
    | llm
    | StrOutputParser()
)

# Teste rápido (opcional)
print(local_info_chain.invoke({"question": "Onde posso visitar no Rio de Janeiro?"}))

No Rio de Janeiro, você pode visitar a Praia de Copacabana, que é ideal para caminhadas no calçadão e visitas a quiosques, especialmente durante o réveillon, quando recebe milhões de turistas. Além disso, o Cristo Redentor é a atração mais famosa da cidade e é recomendado visitá-lo pela manhã, em dias de céu limpo, para aproveitar a vista e evitar filas.


In [3]:
# Prompt Template para Logística
logistics_prompt = PromptTemplate(
    template="""Você é um especialista em logística de viagens.
    Responda à PERGUNTA do usuário focando em transporte, direções e aspectos práticos.
    Use o CONTEXTO para basear sua resposta.
    Se a informação não estiver no contexto, diga que você não sabe. Não invente.

    CONTEXTO: {context}
    PERGUNTA: {question}
    RESPOSTA:""",
    input_variables=["context", "question"],
)

# Criar a Cadeia (Chain)
logistics_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | logistics_prompt
    | llm
    | StrOutputParser()
)

# Teste rápido (opcional)
print(logistics_chain.invoke({"question": "Qual melhor aeroporto para chegar em Paris?"}))

O Aeroporto Charles de Gaulle (CDG) é o principal ponto de entrada internacional em Paris, então é provavelmente a melhor opção para chegar em Paris, especialmente se você estiver vindo de fora da Europa. No entanto, se você estiver vindo de uma região próxima ou dentro da Europa, o Aeroporto Orly (ORY) pode ser uma opção viável, pois é mais utilizado para voos regionais. A escolha do aeroporto depende do seu local de origem e do tipo de voo que você estiver pegando.


In [4]:
itinerary_prompt = PromptTemplate(
    template="""Você é um planejador de viagens experiente.
    Com base na PERGUNTA do usuário e no CONTEXTO de informações turísticas, crie um roteiro de viagem detalhado.
    O roteiro deve ser claro, organizado por dias e períodos (manhã, tarde, noite) e incluir sugestões do contexto.
    Se não for possível formular uma resposta com as informações do contexto, diga que você não sabe. Não invente.

    CONTEXTO: {context}
    PERGUNTA: {question}
    ROTEIRO DETALHADO:""",
    input_variables=["context", "question"],
)

# Criar a Cadeia (Chain)
itinerary_chain = (
    {
        # Extrai o valor da chave "question" e passa SÓ A STRING para o retriever
        "context": itemgetter("question") | retriever,
        # Extrai o valor da chave "question" para repassar ao prompt
        "question": itemgetter("question")
    }
    | itinerary_prompt
    | llm
    | StrOutputParser()
)

# Teste rápido (opcional)
print(itinerary_chain.invoke({"question": "Crie um roteiro de um dia em Paris"}))

Aqui está um roteiro de um dia em Paris, considerando as informações disponíveis no contexto:

**Manhã (9h00 - 12h00)**

* 9h00: Inicie o dia visitando um dos cafés tradicionais parisienses, como sugerido no contexto. Experimente um croissant ou quiche e observe o cotidiano da cidade.
* 10h30: Após o café, faça uma caminhada pelo bairro onde se encontra o café, aproveitando a arquitetura e a atmosfera parisiense.

**Almoço (12h00 - 14h00)**

* 12h00: Para o almoço, sugiro visitar o restaurante Le Meurice, uma referência da alta gastronomia francesa. O ambiente sofisticado e a comida de alta qualidade garantem uma experiência inesquecível.

**Tarde (14h00 - 18h00)**

* 14h00: Após o almoço, faça uma caminhada pelo rio Sena ou visite algum dos principais pontos turísticos da cidade, como a Torre Eiffel ou o Louvre.
* 16h00: Visite uma das muitas lojas ou mercados da cidade, como o Mercado de Antiguidades de Saint-Ouen, para encontrar souvenirs ou itens únicos.

**Noite (18h00 - 22h00)**


In [10]:
translation_prompt = PromptTemplate.from_template(
    """Você é um guia de tradução para turistas.
    O usuário quer saber frases úteis em outro idioma.
    Traduza a seguinte frase ou pedido para o idioma solicitado e forneça uma pronúncia simplificada.

    PEDIDO DO USUÁRIO: {question}
    TRADUÇÃO E GUIA DE PRONÚNCIA:"""
)

# Criar a Cadeia (Chain)
translation_chain = translation_prompt | llm | StrOutputParser()

# Teste rápido (opcional)
print(translation_chain.invoke("Como se diz 'Onde fica o banheiro?' em francês?"))

A tradução para o francês é: "Où sont les toilettes?"

Pronúncia simplificada: "Uh sah lay twah-leh-tuh"

Lembre-se de que, em francês, a pronúncia pode variar ligeiramente dependendo do sotaque e da região, mas essa é uma forma amplamente aceita e compreendida.


In [6]:
from langchain_core.runnables import RunnableBranch

# Template para o classificador
router_prompt_template = """Dada a pergunta do usuário, classifique-a em uma das seguintes categorias.
Responda APENAS com o nome da categoria.

Categorias:
- roteiro-viagem: O usuário está pedindo um itinerário, plano de viagem ou sugestões de atividades para múltiplos dias.
- logistica-transporte: O usuário está perguntando sobre meios de transporte, como chegar a um lugar, aeroportos ou locomoção.
- info-local: O usuário está pedindo informações específicas sobre um ponto turístico, restaurante, horário de funcionamento, etc.
- traducao-idiomas: O usuário está pedindo para traduzir uma frase ou palavra.
- geral: O usuário está fazendo uma pergunta geral que não se encaixa nas outras categorias.

Pergunta do usuário:
{question}

Classificação:"""

prompt_router = PromptTemplate(
    template=router_prompt_template,
    input_variables=["question"],
)

# A cadeia classificadora
classifier_chain = prompt_router | llm | StrOutputParser()

In [7]:
# O RunnableBranch funciona como um if/elif/else
branch = RunnableBranch(
    # Se o tópico for "roteiro-viagem", execute a itinerary_chain
    (lambda x: "roteiro-viagem" in x["topic"].lower(), itinerary_chain),
    # Se o tópico for "logistica-transporte", execute a logistics_chain
    (lambda x: "logistica-transporte" in x["topic"].lower(), logistics_chain),
    # Se o tópico for "info-local", execute a local_info_chain
    (lambda x: "info-local" in x["topic"].lower(), local_info_chain),
    # Se o tópico for "traducao-idiomas", execute a translation_chain
    (lambda x: "traducao-idiomas" in x["topic"].lower(), translation_chain),
    # Se nenhuma condição for atendida, execute a cadeia padrão (LLM puro)
    llm,
)

full_chain = {
    # 1. Adaptamos o input para a classifier_chain:
    #    - Pegamos o dicionário de entrada (ex: {'input': 'texto'})
    #    - Criamos um novo dicionário {'question': 'texto'}
    #    - Passamos esse novo dicionário para a classifier_chain
    "topic": (lambda x: {"question": x["input"]}) | classifier_chain,
    
    # 2. Mantemos a pergunta original para ser usada pelas cadeias finais
    "question": lambda x: x["input"]
} | branch

# Renomeamos a variável final para 'chain' para manter a consistência com os testes
chain = full_chain

In [8]:
# --- Testes ---
print("\n--- Teste de Roteiro ---")
print(chain.invoke({"input": "Me dê um roteiro gastronômico no Rio de Janeiro"}))

print("\n--- Teste de Logística ---")
print(chain.invoke({"input": "Qual a melhor forma de ir do aeroporto Charles de Gaulle para o centro de Paris?"}))


--- Teste de Roteiro ---
Claro, posso ajudar com um roteiro gastronômico no Rio de Janeiro com base nas informações disponíveis. Uma das principais referências em rodízio de carnes brasileiras no Rio de Janeiro é a churrascaria Fogo de Chão. É um ótimo lugar para experimentar a culinária brasileira, especialmente se você gosta de carnes.

Infelizmente, não tenho informações sobre outros restaurantes ou opções gastronômicas específicas no Rio de Janeiro além da churrascaria Fogo de Chão. Se você estiver procurando por uma experiência mais variada, pode ser útil explorar outras fontes ou guias de viagem para obter mais recomendações.

No entanto, se você também estiver interessado em combinar sua experiência gastronômica com atrações turísticas, posso sugerir uma visita à Praia de Copacabana, que é um ponto turístico famoso na cidade e oferece um calçadão para caminhadas e quiosques para relaxar. Durante o réveillon, é um local especialmente animado, atraindo milhões de turistas.

Esper

In [9]:
# --- Bloco de Testes Opcional para Cadeias Individuais ---

print("\n" + "="*50)
print("INICIANDO TESTES INDIVIDUAIS DAS CADEIAS")
print("="*50 + "\n")

# 1. Teste da Itinerary Chain (RAG)
print("--- Testando: Itinerary Chain ---")
try:
    # A entrada agora é um dicionário com a chave 'question'
    response = itinerary_chain.invoke({"question": "Crie um roteiro cultural de 2 dias no Rio de Janeiro"})
    print(response)
except Exception as e:
    print(f"Erro ao testar Itinerary Chain: {e}")
print("\n" + "-"*50 + "\n")


# 2. Teste da Logistics Chain (RAG)
print("--- Testando: Logistics Chain ---")
try:
    response = logistics_chain.invoke({"question": "Como chego no Cristo Redentor partindo de Copacabana?"})
    print(response)
except Exception as e:
    print(f"Erro ao testar Logistics Chain: {e}")
print("\n" + "-"*50 + "\n")


# 3. Teste da Local Info Chain (RAG)
print("--- Testando: Local Info Chain ---")
try:
    response = local_info_chain.invoke({"question": "Qual o horário de funcionamento do Museu do Louvre?"})
    print(response)
except Exception as e:
    print(f"Erro ao testar Local Info Chain: {e}")
print("\n" + "-"*50 + "\n")


# 4. Teste da Translation Chain (Sem RAG, mas também espera 'question')
# O prompt dela também usa a variável {question}
print("--- Testando: Translation Chain ---")
try:
    response = translation_chain.invoke({"question": "Como se diz 'bom dia' em francês?"})
    print(response)
except Exception as e:
    print(f"Erro ao testar Translation Chain: {e}")
print("\n" + "="*50)
print("TESTES INDIVIDUAIS CONCLUÍDOS")
print("="*50 + "\n")


INICIANDO TESTES INDIVIDUAIS DAS CADEIAS

--- Testando: Itinerary Chain ---
Aqui está um roteiro cultural de 2 dias no Rio de Janeiro:

**Dia 1:**

* **Manhã:** Inicie o dia com um café da manhã leve em um dos quiosques da Praia de Copacabana. Aproveite a vista do mar e o movimento do calçadão.
* **Tarde:** Após o café da manhã, faça uma caminhada no calçadão da Praia de Copacabana. Aproveite a arquitetura dos prédios ao longo da praia e a vista do Pão de Açúcar.
* **Noite:** Para jantar, vá à churrascaria Fogo de Chão, uma referência em rodízio de carnes brasileiras. Aproveite a variedade de carnes e a atmosfera aconchegante do restaurante.

**Dia 2:**

* **Manhã:** Passe a manhã explorando o bairro de Santa Teresa, conhecido por suas ruas de paralelepípedos e arquitetura colonial. Visite o Museu Chácara do Céu e admire a vista da cidade.
* **Tarde:** Após o almoço, vá ao bairro de Lapa, conhecido por sua vida noturna e pontos turísticos como o Arco do Teles e a Escadaria Selarón.
* 